### BCO-DMO Knowledge Graph Data Exploration Prototype
This is a prototype demonstrating how python can be used to interactively explore oceanographic data within the BCO-DMO Knowledge Graph. This demonstration was developed for SciPy 2020. 

**WARNING** This is just a prototype and will likely be updated (or abandoned ¯\\_(ツ)_/¯). In addition, the BCO-DMO Knowledge Graph is also under construction, so stability of this prototype is far from guaranteed. This is mainly just an example of how we might leverage the Knowledge Graph to facilitate interactive exploration of oceanographic data. Hope to have some amazing (and stable) tools for exploration of the Graph in the future. Brutal honesty moment: This tool which was just intended for a viz example has been great for revealing some issues with tagging of data in the Graph -- side bonus for us at BCO-DMO so we can fix these, but this does result in some issues visualizing some datasets.

**WARNING \#2:** Some of the datasets within BCO-DMO are very large. Therefore, for performance reasons, a limit on datasets displayed is set below. Feel free to change.

In [1]:
MAX_DATASET_SHOW = 5

In [2]:
from bqplot import Lines, Figure, LinearScale, DateScale, Axis
from ipyleaflet import Map, GeoJSON, basemaps, WidgetControl, Marker, MarkerCluster
from ipywidgets import link, HTML
import json
import os
import sys
import requests
import geopandas
import pandas as pd
import numpy as np
import rdflib
from SPARQLWrapper import SPARQLWrapper, JSON
from ipywidgets import Layout, IntText, Dropdown, Combobox, VBox, IntSlider

In [3]:
#credit: Doug Fils
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']
    
    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            #item.append(str(row.get(c, {}).get('value')))
            item.append(row.get(c, {}).get('value'))
        out.append(item)
# could simply return 'out' which is a list of lists
    return pd.DataFrame(out, columns=cols)

In [4]:
BCODMO_SERVE = "https://lod.bco-dmo.org/sparql"  #BCO-DMO SPARQL Endpoint
BCODMO_PREF = "http://lod.bco-dmo.org/id/"       #BCO-DMO URI prefix

In [5]:
## Paramter options
###### Dataset Description Widget
#SPARQL query for BCO-DMO dataset information
masterParameterQuery = """
SELECT DISTINCT ?masterParamtersId ?shortDesc ?label #?datasetParameter #?url
WHERE {
    <http://lod.bco-dmo.org/id/parameters> ?property ?masterParamtersId .
    ?masterParamtersId owl:deprecated "0"^^xsd:boolean . #remove deprecated master parameters
    ?masterParamtersId odo:hasParameterShortDescription ?shortDesc .
    ?masterParamtersId skos:prefLabel ?label .
    ?datasetParameter odo:isInstanceOf ?masterParametersId .
    ?dataset odo:storesValuesFor ?datasetParameter .
    #Select only those master params that have depths
    ?datasetParameter odo:isInstanceOf <http://lod.bco-dmo.org/id/parameter/808> . 
    #end
    ?affordance schema:subjectOf ?dataset .
    ?affordance rdf:type ?action_type .
    ?affordance schema:target ?target .
    ?target schema:contentType "application/geo+json"^^xsd:token .
    #?target schema:url ?url .
}
ORDER BY ?shortDesc ?label ?masterParametersId
"""
df_masterParams_with_geoJson = get_sparql_dataframe(BCODMO_SERVE, masterParameterQuery)


In [6]:
#Displays dropdown selector for dataset parameters

masterParamsOptions = df_masterParams_with_geoJson["label"].values.tolist()
style = {'description_width': 'initial'}
masterParamsMenu = Combobox(
    options=masterParamsOptions,
    description='Search Parameter:',
    disabled=False,
    layout=Layout(width='80%'),
    continuous_update=False,
    style=style
)

###### Update Dataset Parameter Description with Menu Selection
def handle_masterParam_change(change):
    if change.new != change.old:
        masterParamsMenu.value = change.new

masterParamsMenu.observe(handle_masterParam_change, names='value') #observer for change       
       
masterParamsMenu

Combobox(value='', continuous_update=False, description='Search Parameter:', layout=Layout(width='80%'), optio…

In [7]:
parameterSelected = df_masterParams_with_geoJson["masterParamtersId"].loc[df_masterParams_with_geoJson["label"]\
                                == masterParamsMenu.value]
parameterSelected = parameterSelected.to_string(index=False).strip()

In [8]:
###Select Nitrite and see what happens - http://lod.bco-dmo.org/id/parameter/1192

nitriteQuery = """
SELECT DISTINCT ?masterParam ?nanValue ?unit ?parameterName ?datasetID ?url
WHERE {
    VALUES ?masterParam {<""" + parameterSelected + """>}
    ?dataset_parameter odo:isInstanceOf ?masterParam .
    ?dataset_parameter odo:hasNoDataValue ?nanValue .
    ?dataset_parameter odo:hasUnitOfMeasure ?nodeUnit .
    ?nodeUnit rdf:value ?unit .
    ?dataset_parameter skos:prefLabel ?parameterName .
    ?dataset odo:storesValuesFor ?dataset_parameter . 
    #?dataset_parameter odo:isInstanceOf <http://lod.bco-dmo.org/id/parameter/808> .
    ?dataset dcterms:identifier ?datasetID .
    #check GeoJSON
    ?affordance schema:subjectOf ?dataset .
    ?affordance rdf:type ?action_type .
    ?affordance schema:target ?target .
    ?target schema:contentType "text/csv"^^xsd:token .
    ?target schema:url ?url .
}
"""
df_parameter = get_sparql_dataframe(BCODMO_SERVE, nitriteQuery)
#df_parameter

In [9]:
# All the datasets that have the target parameter
listDataSetIDs = df_parameter["datasetID"].astype("str").values.tolist()
listDataSetIDsStr = ' '.join(listDataSetIDs)

In [10]:
nitriteDepthQuery = """
SELECT DISTINCT ?masterParamDepth ?nanValueDepth ?unitDepth ?col_nameDepth ?datasetID ?url
WHERE {
    VALUES ?datasetID {""" + listDataSetIDsStr + """} 
    VALUES ?masterParamDepth { <http://lod.bco-dmo.org/id/parameter/808>} #808 is the parameter for Depth
    ?dataset_parameter odo:isInstanceOf ?masterParamDepth .
    ?dataset_parameter odo:hasNoDataValue ?nanValueDepth .
    ?dataset_parameter odo:hasUnitOfMeasure ?nodeUnit .
    ?nodeUnit rdf:value ?unitDepth .
    ?dataset_parameter skos:prefLabel ?col_nameDepth .
    ?dataset odo:storesValuesFor ?dataset_parameter . 
    #?dataset_parameter odo:isInstanceOf <http://lod.bco-dmo.org/id/parameter/808> .
    ?dataset dcterms:identifier ?datasetID .
    #check GeoJSON
    ?affordance schema:subjectOf ?dataset .
    ?affordance rdf:type ?action_type .
    ?affordance schema:target ?target .
    ?target schema:contentType "text/csv"^^xsd:token .
    ?target schema:url ?url .
}
"""
df_parameterDepth = get_sparql_dataframe(BCODMO_SERVE, nitriteDepthQuery)
#parameterDepth_df

In [11]:
#df_parameterDepth.style.set_properties(subset=['url'], **{'width': '600px'})

In [12]:
#Find all datasets that have the target parameter and associated depth data
df_dataSetsWithParameterAndDepth = df_parameter.loc[df_parameter["datasetID"].isin(df_parameterDepth["datasetID"].unique())].reset_index()

In [13]:
df_dataSetsWithParameterAndDepth.drop_duplicates(subset="url", inplace=True)
urlTest = df_dataSetsWithParameterAndDepth[["url", "datasetID"]].values[0:MAX_DATASET_SHOW] #limiting to 5 datasets max right now

In [14]:
#create groupbys on specific datasets
df_parameterDepth["parameterType"] = "depth"
try:
    df_dataSetsWithParameterAndDepth = df_dataSetsWithParameterAndDepth.drop(columns=["index"])
except:
    print("no column named index")
    
df_dataSetsWithParameterAndDepth["parameterType"] = str(masterParamsMenu.value)
df_parameterDepth = df_parameterDepth.rename(columns={"masterParamDepth":"masterParam", \
                                                      "col_nameDepth":"parameterName", \
                                                      "unitDepth":"unit", \
                                                      "nanValueDepth":"nanValue"})
df_paramsAndDepths = pd.concat([df_parameterDepth, df_dataSetsWithParameterAndDepth])


gb_paramDepth = df_paramsAndDepths.groupby("datasetID")

In [15]:
dfg_big = geopandas.GeoDataFrame(columns=['datasetID', 'geometry', 'parameterName', 'value', 'parameterType', 'unit'])
dfg_points = geopandas.GeoDataFrame(columns=["datasetID", "geometry"])

for url, datasetID in urlTest:

    #generate lists of parameters and depth dataset-specific column names    
    subdf = gb_paramDepth.get_group(datasetID)
    paramCols = subdf["parameterName"].loc[subdf["parameterType"] == masterParamsMenu.value].unique().tolist()
    depthCols = subdf["parameterName"].loc[subdf["parameterType"] == "depth"].unique().tolist()
    depthCols = depthCols + ["depth"]
    #Adding some common nan issues -- need to update KG where there are multiple NaNs/dataset parameter
    nanValues = subdf["nanValue"].unique().tolist() + ["n.a.", "nan", "-9999", "-999.0", "-999", "mix", ""]# \
                                                      # "bdl", 'Below_detection_limit', 'ND', 'DNP', 'BDL']# coerce all these fun strings in float cols
    df_units = subdf[["unit", "parameterName"]]
    
    data = pd.read_csv(url, low_memory=False)
    data = data.drop([0])
    
    colsKeep = ["latitude", "longitude"] + depthCols + paramCols
    checkParamInFile = all(item in data.columns for item in colsKeep)
    if checkParamInFile is False:
        colsKeep = [s for s in colsKeep if s in data.columns]
        paramCols = [s for s in paramCols if s in data.columns]
        depthCols = [s for s in depthCols if s in data.columns]
    #Drop dataset if latitude or longitude don't exist
    checkCoords = all(item in data.columns for item in ["latitude", "longitude"])
    if checkCoords is False:
        continue
        
    dfg = data[colsKeep].copy() 
    dfg["datasetID"] = datasetID 
    [dfg.replace(x, np.nan, inplace=True) for x in nanValues]
    
    #add unique subset of location points to points dataframe for mapping
    dfg["longitude"] = dfg["longitude"].astype("float")
    dfg["latitude"] = dfg["latitude"].astype("float")
    
    dfg_geometry = geopandas.GeoDataFrame(dfg, geometry=geopandas.points_from_xy(dfg["longitude"], dfg["latitude"]))
    
    #Drop rows that don't have data for the selected parameter
    dfg_geometry[paramCols] = dfg_geometry[paramCols].fillna(1).apply(lambda x: pd.to_numeric(x, errors='coerce'))
    dfg_geometry = dfg_geometry.dropna(subset=paramCols)
    
    dfg_points = dfg_points.append(dfg_geometry[["datasetID", "geometry"]].drop_duplicates())
    
    paramsList = paramCols + depthCols
    dfg_melt = pd.melt(dfg_geometry, id_vars=["datasetID", "geometry"], \
             value_vars=[c for c in dfg_geometry.columns if c in paramsList],\
            var_name='parameterName')
    dfg_melt["parameterType"] = "depth"
    dfg_melt.loc[dfg_melt["parameterName"].isin(paramCols), ['parameterType']] = 'parameter' 
    dfg_melt = pd.merge(dfg_melt, df_units, how="left")
    
    dfg_big = dfg_big.append(dfg_melt)     

#convert dfg_points to geoJson
dfg_points = dfg_points.reset_index().drop(columns='index')
point_geoJson = dfg_points.to_file("points.geojson", driver='GeoJSON')

with open('points.geojson', 'r') as f:
    data = json.load(f)

In [16]:
dfp = pd.DataFrame(dfg_big) #convert to pandas dataframe to do more 
dfp["geometry_str"] = dfp["geometry"].astype("str").str.replace("POINT ", "").str.replace("(", "").str.replace(")", "")
dfp[['longitude','latitude']] = dfp["geometry_str"].str.split(expand=True)
dfp["value"] = dfp["value"].astype("float").round(2)
dfp["value"].loc[dfp["parameterName"] == masterParamsMenu.value].astype("float").round(0)

dfp[['longitude','latitude']] = dfp[['longitude','latitude']].astype("float")
#dfp[['longitude','latitude']] = dfp[['longitude','latitude']].round(5)
dfp["lon_lat"] = dfp["latitude"].astype("str") + " " + dfp["longitude"].astype("str")


In [ ]:
gb_dfp = dfp.groupby(["datasetID"])    
                                                               
m = Map(center=(0, 0), zoom=1, basemap=basemaps.Esri.NatGeoWorldMap)
geo_json = GeoJSON(data=data, style = {
})

m.add_layer(geo_json)

html1 = HTML('''
    <h4>Dataset Info</h4>
    Click on a point
''')
html1.layout.margin = '0px 20px 20px 20px'
control1 = WidgetControl(widget=html1, position='bottomleft')


def update_html(feature, **kwargs):
    html1.value = '''
        <b>Dataset: {}</b></br>
        <a>https://www.bco-dmo.org/dataset/{}</a>
    '''.format(feature['properties']['datasetID'], feature['properties']['datasetID'])

geo_json.on_click(update_html)

#add minimap
#minimap = Map(
#    zoom_control=True, attribution_control=False, 
#    zoom=-2, center=m.center, basemap=basemaps.Esri.WorldImagery 
#)
#minimap.layout.width = '250px'
#minimap.layout.height = '200px'
#### Changed the datatype of dfg_points, so would need to update this in order for it to work
#minimap.add_layer(MarkerCluster(markers=[Marker(location=geolocation.coords[0][::-1]) \
#                                         for geolocation in dfg_points.geometry.unique()]))
#link((minimap, 'center'), (m, 'center'))
#minimap_control = WidgetControl(widget=minimap, position='bottomleft')

#m.add_control(minimap_control)
m.add_control(control1)

#m

In [ ]:
x_data = []
depth_data = []

x_sc = LinearScale()
y_sc = LinearScale(reverse=True)

line = Lines(x=x_data,
             y=depth_data,
             scales={'x': x_sc, 'y': y_sc},
             colors=['orange', 'red', 'blue', 'black'])

ax_x = Axis(label="", scale=x_sc, tick_format='0.1f', num_ticks=5)
ax_y = Axis(label="", scale=y_sc,
            orientation='vertical', tick_format='0.0f', side='left')

figure = Figure(axes=[ax_x, ax_y], marks=[line], animation_duration=300,
                layout={'max_height': '270px'}, title=masterParamsMenu.value)


#Make the Widgets for selecting parameters
paramOptions = [""]
style = {'description_width': 'initial'}
paramPlotMenu = Dropdown(
    options=paramOptions,
    description='Parameter to plot:',
    disabled=False,
    layout=Layout(width='80%'),
    continuous_update=False,
    value="",
    style=style
)

depthOptions = [""]
style = {'description_width': 'initial'}
depthPlotMenu = Dropdown(
    options=depthOptions,
    description='Depth to plot:',
    disabled=False,
    layout=Layout(width='80%'),
    continuous_update=False,
    value="",
    style=style
)

figureDisplay = VBox([figure, paramPlotMenu, depthPlotMenu])
#figureDisplay

In [ ]:
def update_figure(datasetID, dfp_paramType, gb_dfp_sub_point):
    
    paramPlotMenu.options = dfp_paramType["parameterName"].loc[dfp_paramType["parameterType"] == "parameter"].unique()
    depthPlotMenu.options = dfp_paramType["parameterName"].loc[dfp_paramType["parameterType"] == "depth"].unique()
        
    def paramSelect_changed(change):
        if change.new != change.old:
            paramPlotMenu.value = change.new
    
    
    def depthSelect_changed(change):
        if change.new != change.old:
            depthPlotMenu.value = change.new
    
    paramPlotMenu.observe(paramSelect_changed, names='value')
    depthPlotMenu.observe(depthSelect_changed, names='value')

    parameter = gb_dfp_sub_point[paramPlotMenu.value].dropna().values
    depth = gb_dfp_sub_point[depthPlotMenu.value].dropna().values
    if len(parameter) == len(depth): #Need at least 2 points for a depth profile
        line.x = parameter
        line.y = depth
        figure.title = paramPlotMenu.value
        ax_x.label = dfp_paramType["unit"].loc[dfp_paramType["parameterName"] == paramPlotMenu.value].to_string(index=False)
        ax_y.label = dfp_paramType["unit"].loc[dfp_paramType["parameterName"] == depthPlotMenu.value].to_string(index=False)
    else:
        figure.title = "Incompatible parameter & depth"
        line.x = [0]
        line.y = [0]

In [ ]:
widget_control1 = WidgetControl(widget=figureDisplay, position='topright')
m.add_control(widget_control1)

In [ ]:
def plot_on_click(event, feature, **kwargs):
    global datasetID, dfp_paramType, gb_dfp_sub_point
    
    coordsList = feature["geometry"]["coordinates"]

    datasetID = feature["properties"]["datasetID"]
    #point = str('%.5f' % coordsList[1]) + " " + str('%.5f' % coordsList[0])
    point = str(coordsList[1]) + " " + str(coordsList[0])

    
    dfp_sub = gb_dfp.get_group(str(datasetID))
    dfp_paramType = dfp_sub[["parameterType", "parameterName", "unit"]].drop_duplicates()
    
    dfp_sub = dfp_sub.drop(columns=["unit", "parameterType", "geometry", "datasetID", \
                                    "longitude", "latitude"])
    #print(dfp_sub)
    gb_dfp_sub = dfp_sub.groupby(["lon_lat"])
    #print(gb_dfp_sub.groups)
    gb_dfp_sub_point = gb_dfp_sub.get_group(point).reset_index()
    gb_dfp_sub_point = gb_dfp_sub_point.drop(columns=["lon_lat"])
    gb_dfp_sub_point = gb_dfp_sub_point.pivot(index=None, columns='parameterName', values=["value"])
    gb_dfp_sub_point.columns = gb_dfp_sub_point.columns.droplevel(level=0)
    
    update_figure(datasetID, dfp_paramType, gb_dfp_sub_point)#add back point

geo_json.on_click(plot_on_click)

In [ ]:
m